![StatModels](https://www.durhamtech.edu/themes/custom/durhamtech/images/durham-tech-logo-web.svg) 

## Applications - Statistical Modeling

This lecture provides foundational knowledge and examples of machine learning modeling concepts by examining stock price data.

---

# Table of Contents

### Jupyter Overview
#### <a href='#1'>Useful Links</a>
#### <a href='#2'>Introduction to Jupyter Notebooks</a>
#### <a href='#3'>Cell Types</a>
* Markdown 
* Code
    1. Running One Cell
    2. Other Run Options

#### <a href='#4'>Tips and Tricks</a>

#### <a href='#55'>Weekly Readings/Videos</a>
#### <a href='#56'>Extra Practice</a>

## Introduction

![FunnyML](https://www.meme-arsenal.com/memes/11f11b5d16eef661677e4c9e989dd2b3.jpg) 

## Data Sources

1. Weather: https://www.weather.gov/wrh/climate?wfo=okx
2. SP 500 Components: https://datahub.io/core/s-and-p-500-companies
3. SP 500 Company Info: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
4. FRED https://fred.stlouisfed.org/docs/api/fred/series.html
5. TD Ameritrade Data Dicitionary https://developer.tdameritrade.com/content/streaming-data#_Toc504640567



In [1]:
# https://pypi.org/project/yahoo-finance-api2/
# https://github.com/pkout/yahoo_finance_api2

# Uncomment below if you don't have yahoo finance api installed
# pip install yahoo_finance_api2

In [2]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import time
import math
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
import warnings
import matplotlib.pyplot as plt
import numpy as np
import json
import datetime

warnings.filterwarnings('ignore')
key = 'RGOLSJPSTGVAN4NTN4DLWJE71SU7SIH0'

In [3]:
# view file contents
%ls

DFF.csv                    SP500_info.csv
Data_Cleaning-Copy1.ipynb  UNRATE.csv
Data_Cleaning.ipynb        constituents_csv.csv
GDPC1.csv                  nyc_precip.xlsx
Machine_Learning.ipynb     nyc_temp.xlsx
Regression.ipynb


In [4]:
tickers=pd.read_csv("constituents_csv.csv")
print(len(tickers))
tickers.head()

505


,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care


In [5]:
ticker_info=pd.read_csv("sp500_info.csv")
print(len(ticker_info))
ticker_info.head()

505


,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",8/9/76,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/64,1800,1888
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/12,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/18,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/11,1467373,1989


In [6]:
tickers=pd.merge(tickers,ticker_info,on='Symbol',how='inner')
print(len(tickers))
tickers.head()

505


,Symbol,Name,Sector,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,Industrials,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",8/9/76,66740,1902
1,AOS,A. O. Smith,Industrials,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",7/26/17,91142,1916
2,ABT,Abbott Laboratories,Health Care,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/64,1800,1888
3,ABBV,AbbVie,Health Care,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/12,1551152,2013 (1888)
4,ABMD,Abiomed,Health Care,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/18,815094,1981


In [7]:
del ticker_info

In [8]:
def get_td_price_hist(ticker,period,key,row_count='Blank'):
    time.sleep(1)
    endpoint = 'https://api.tdameritrade.com/v1/marketdata/'+ticker+'/pricehistory'

    ##Define Payload
    payload = {'apikey': key,
    'periodType': 'year',
    'period':period,
    'frequencyType':'daily'}

    ### make request
    try:
        content = requests.get(url = endpoint, params = payload)
    except:
        print('API error, please review.')
        
    ### Convert to dictionary
    dictlist = []
    data = content.json()

    for key, value in data.items():
        temp = [key,value]
        dictlist.append(temp)
        
    try:
        hist_data = pd.DataFrame(dictlist[0][1])
        hist_data['datetime'] = pd.to_datetime(hist_data['datetime'],unit='ms')
        hist_data.sort_values(by=['datetime'],ascending=False)
        hist_data=hist_data.sort_values(by=['datetime'],ascending=True).reset_index()
        hist_data['Date']=hist_data['datetime'].dt.date
        hist_data=hist_data.drop(['index','datetime'],axis=1)
        hist_data['ticker'] = ticker
        if row_count!='Blank':
            return hist_data.tail(row_count)
        else:
            return hist_data
    except:
        df = pd.DataFrame()
        print('running except clause')
        return df
    
def get_fundamental_from_td(ticker,key):
    time.sleep(1)
    endpoint = 'https://api.tdameritrade.com/v1/instruments'
    projection = 'fundamental'

    ##Define Payload
    payload = {'apikey': key,
               'symbol' : ticker,
                'projection': projection,
                }
    
    ### make request
    try:
        content = requests.get(url = endpoint, params = payload)
    except:
        print('API error, please review.')
        
    ### Convert to dictionary
    dictlist = []
    data = content.json()
    for key, value in data.items():
        temp = [key,value]
        dictlist.append(temp)
        
    try:
        df = pd.DataFrame(dictlist[0][1]).T.reset_index(drop=True).iloc[0]
        return df
    except:
        print(dictlist)
        df = pd.DataFrame()
        print(ticker + " not valid.")
        return df
    
def get_yahoo_history(share_name):
    print("Pulling history")

    if share_name[-1]=='2':
        my_share = share.Share(share_name[:-1])
    else:
        my_share = share.Share(share_name)
    symbol_data = None

    try:
        symbol_data = my_share.get_historical(share.PERIOD_TYPE_YEAR,
                                              30000,
                                              share.FREQUENCY_TYPE_DAY,
                                              1)
        df = pd.DataFrame(symbol_data)
        df['timestamp'] = df['timestamp'].astype(str)
        df['timestamp'] = df['timestamp'].map(lambda x: x[:-3])
        df['Date'] =df['timestamp'].astype('int')
        df['Date'] = pd.to_datetime(df['Date'],unit='s')
        df=df.sort_values(by=['Date'],ascending=True)
        df['Date']=df['Date'].dt.date
        df=df.drop(['timestamp'],axis=1)
        df['ticker']=str(share_name.upper())
    except YahooFinanceError as e:
        print(e.message)
        sys.exit(1)
    return df

In [9]:
ticker=tickers['Symbol'][1]

In [10]:
tickers.iloc[[1]]

,Symbol,Name,Sector,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
1,AOS,A. O. Smith,Industrials,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",7/26/17,91142,1916


In [11]:
pd.DataFrame(get_fundamental_from_td(ticker,key)).T

,beta,bookValuePerShare,currentRatio,divGrowthRate3Year,dividendAmount,dividendDate,dividendPayAmount,dividendPayDate,dividendYield,epsChange,...,revChangeYear,sharesOutstanding,shortIntDayToCover,shortIntToFloat,symbol,totalDebtToCapital,totalDebtToEquity,vol10DayAvg,vol1DayAvg,vol3MonthAvg
0,1.15013,5.59895,1.76888,0,1.12,2021-10-28 00:00:00.000,0.28,2021-11-15 00:00:00.000,1.36,0,...,0,1.33187e+08,0,0,AOS,5.3123,5.61033,737038,737040,2.28605e+07


In [12]:
print(ticker)
#td_data=get_td_price_hist(ticker,1,key,43)
td_data=get_td_price_hist(ticker,1,key)
print(len(td_data))
td_data.tail()

AOS
253


,open,high,low,close,volume,Date,ticker
248,82.68,82.96,81.8100,82.31,744896,2021-11-18,AOS
249,82.52,83.04,81.6505,82.46,726617,2021-11-19,AOS
250,82.88,83.48,82.3900,82.80,673088,2021-11-22,AOS
251,82.99,83.39,81.8710,82.45,620493,2021-11-23,AOS
252,82.16,82.93,81.9300,82.39,846838,2021-11-24,AOS


In [13]:
yahoo_data=get_yahoo_history(ticker)
print(len(yahoo_data))
yahoo_data.tail()

Pulling history
9620


,open,high,low,close,volume,Date,ticker
9615,82.519997,83.040001,81.650002,82.459999,726600,2021-11-19,AOS
9616,82.879997,83.480003,82.389999,82.800003,673100,2021-11-22,AOS
9617,82.989998,83.389999,81.870003,82.449997,620500,2021-11-23,AOS
9618,82.160004,82.930000,81.930000,82.389999,846800,2021-11-24,AOS
9619,80.919998,81.760002,80.339996,81.000000,638640,2021-11-26,AOS


In [14]:
#pd.merge(yahoo_data,td_data,on=['Date'],how='outer',indicator=True)
pd.merge(yahoo_data.tail(10),td_data.tail(10),on=['Date'],how='outer',indicator=True).describe()

,open_x,high_x,low_x,close_x,volume_x,open_y,high_y,low_y,close_y,volume_y
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,82.037999,82.835001,81.459999,82.252999,734264.000000,81.833000,82.645700,81.285150,82.120000,737038.200000
std,1.002162,0.630877,0.817979,0.533418,116710.216938,1.390644,1.096741,1.187883,0.912006,114498.036192
min,79.779999,81.760002,79.779999,81.000000,542200.000000,78.870000,79.875000,78.590000,79.670000,542178.000000
25%,81.857498,82.420000,81.207499,82.144997,647255.000000,81.857500,82.417750,81.207625,82.145000,668087.000000
50%,82.340000,82.950001,81.809998,82.419998,735750.000000,82.340000,82.950000,81.810000,82.420000,735756.500000
75%,82.670000,83.302500,81.915001,82.504997,830825.000000,82.670000,83.302500,81.915250,82.505000,830842.500000
max,82.989998,83.599998,82.389999,82.800003,891600.000000,82.990000,83.595000,82.390000,82.800000,891616.000000


In [15]:
ticker='^GSPC'

In [16]:
yahoo_data=get_yahoo_history(ticker)
print(len(yahoo_data))
yahoo_data

Pulling history
14843


,open,high,low,close,volume,Date,ticker
0,0.000000,62.580002,61.720001,62.320000,3700000,1962-12-11,^GSPC
1,0.000000,63.160000,62.130001,62.630001,3760000,1962-12-12,^GSPC
2,0.000000,63.070000,62.090000,62.419998,3380000,1962-12-13,^GSPC
3,0.000000,62.830002,61.959999,62.570000,3280000,1962-12-14,^GSPC
4,0.000000,62.950001,62.139999,62.369999,3590000,1962-12-17,^GSPC
...,...,...,...,...,...,...,...
14838,4708.439941,4717.750000,4694.220215,4697.959961,3265600000,2021-11-19,^GSPC
14839,4712.000000,4743.830078,4682.169922,4682.939941,3206280000,2021-11-22,^GSPC
14840,4678.479980,4699.390137,4652.660156,4690.700195,3428780000,2021-11-23,^GSPC
14841,4675.779785,4702.870117,4659.890137,4701.459961,2464040000,2021-11-24,^GSPC


In [17]:
temp=pd.read_excel('nyc_temp.xlsx')
prec=pd.read_excel('nyc_precip.xlsx')
temp

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1960,33.9,36.3,33.4,54.0,62.6,71.8,74.5,74.9,68.0,58.0,49.7,30.9
1,1961,27.7,36.6,41.5,49.0,59.8,72.4,78.1,76.4,73.5,61.1,48.7,35.5
2,1962,32.6,31.8,43.0,53.3,64.4,72.5,74.0,72.4,64.9,57.4,43.2,31.5
3,1963,30.1,28.2,43.7,53.7,61.1,70.9,76.4,72.1,63.1,61.8,50.3,31.2
4,1964,35.6,32.8,43.1,49.7,65.4,71.6,75.4,72.9,67.1,55.0,49.4,36.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2017,38.0,41.6,39.2,57.2,61.1,72.0,76.8,74.0,70.5,64.1,46.6,35
58,2018,31.7,42.0,40.1,49.5,66.9,71.7,77.6,78.1,70.7,57.7,44.4,40.1
59,2019,32.5,36.2,41.7,55.5,62.2,71.7,79.6,75.5,70.4,59.9,43.9,38.3
60,2020,39.1,40.1,48.0,50.4,60.3,73.7,80.0,76.9,68.8,57.9,53.0,39.2


In [18]:
prec=prec.replace('', np.nan).set_index('Year').stack().reset_index(name='prec').rename(columns={"level_1": "Month"})
temp=temp.replace('', np.nan).set_index('Year').stack().reset_index(name='temp').rename(columns={"level_1": "Month"})
temp

,Year,Month,temp
0,1960,Jan,33.9
1,1960,Feb,36.3
2,1960,Mar,33.4
3,1960,Apr,54
4,1960,May,62.6
...,...,...,...
739,2021,Aug,77.5
740,2021,Sep,70.3
741,2021,Oct,62
742,2021,Nov,47.9


In [19]:
# https://stackoverflow.com/questions/42684530/convert-a-column-in-a-python-pandas-from-string-month-into-int
from calendar import month_abbr

lower_ma = [m.lower() for m in month_abbr]

prec['Month']=prec['Month'].str.lower().map(lambda m: lower_ma.index(m)).astype('Int8')
temp['Month']=temp['Month'].str.lower().map(lambda m: lower_ma.index(m)).astype('Int8')


yahoo_data['month']=pd.to_datetime(yahoo_data.Date).dt.month
yahoo_data['year']=pd.to_datetime(yahoo_data.Date).dt.year
yahoo_data['quarter']=pd.to_datetime(yahoo_data.Date).dt.quarter
yahoo_data=pd.merge(yahoo_data,prec,left_on=['month','year'],right_on=['Month','Year'],how='left')
yahoo_data=yahoo_data.drop(['Month','Year'],axis=1)
yahoo_data=pd.merge(yahoo_data,temp,left_on=['month','year'],right_on=['Month','Year'],how='left')
yahoo_data=yahoo_data.drop(['Month','Year'],axis=1)
yahoo_data

,open,high,low,close,volume,Date,ticker,month,year,quarter,prec,temp
0,0.000000,62.580002,61.720001,62.320000,3700000,1962-12-11,^GSPC,12,1962,4,2.26,31.5
1,0.000000,63.160000,62.130001,62.630001,3760000,1962-12-12,^GSPC,12,1962,4,2.26,31.5
2,0.000000,63.070000,62.090000,62.419998,3380000,1962-12-13,^GSPC,12,1962,4,2.26,31.5
3,0.000000,62.830002,61.959999,62.570000,3280000,1962-12-14,^GSPC,12,1962,4,2.26,31.5
4,0.000000,62.950001,62.139999,62.369999,3590000,1962-12-17,^GSPC,12,1962,4,2.26,31.5
...,...,...,...,...,...,...,...,...,...,...,...,...
14838,4708.439941,4717.750000,4694.220215,4697.959961,3265600000,2021-11-19,^GSPC,11,2021,4,M,47.9
14839,4712.000000,4743.830078,4682.169922,4682.939941,3206280000,2021-11-22,^GSPC,11,2021,4,M,47.9
14840,4678.479980,4699.390137,4652.660156,4690.700195,3428780000,2021-11-23,^GSPC,11,2021,4,M,47.9
14841,4675.779785,4702.870117,4659.890137,4701.459961,2464040000,2021-11-24,^GSPC,11,2021,4,M,47.9


In [20]:
yahoo_data[(yahoo_data['prec']=='M')|(yahoo_data['temp']=='M')]

,open,high,low,close,volume,Date,ticker,month,year,quarter,prec,temp
14824,4610.620117,4620.339844,4595.060059,4613.669922,2924000000,2021-11-01,^GSPC,11,2021,4,M,47.9
14825,4613.339844,4635.149902,4613.339844,4630.649902,3309690000,2021-11-02,^GSPC,11,2021,4,M,47.9
14826,4630.649902,4663.459961,4621.189941,4660.569824,3339440000,2021-11-03,^GSPC,11,2021,4,M,47.9
14827,4662.930176,4683.000000,4662.589844,4680.060059,3332940000,2021-11-04,^GSPC,11,2021,4,M,47.9
14828,4699.259766,4718.500000,4681.319824,4697.529785,3491150000,2021-11-05,^GSPC,11,2021,4,M,47.9
14829,4701.479980,4714.919922,4694.390137,4701.700195,3465720000,2021-11-08,^GSPC,11,2021,4,M,47.9
14830,4707.250000,4708.529785,4670.870117,4685.250000,3110230000,2021-11-09,^GSPC,11,2021,4,M,47.9
14831,4670.259766,4684.850098,4630.859863,4646.709961,3581630000,2021-11-10,^GSPC,11,2021,4,M,47.9
14832,4659.390137,4664.549805,4648.310059,4649.270020,2623140000,2021-11-11,^GSPC,11,2021,4,M,47.9
14833,4655.240234,4688.470215,4650.770020,4682.850098,2865790000,2021-11-12,^GSPC,11,2021,4,M,47.9


In [21]:
dff=pd.read_csv('DFF.csv')

unrate=pd.read_csv('UNRATE.csv')
unrate['month']=pd.to_datetime(unrate.DATE).dt.month
unrate['year']=pd.to_datetime(unrate.DATE).dt.year
unrate.drop('DATE',axis=1,inplace=True)

gdp=pd.read_csv('GDPC1.csv')
gdp['quarter']=pd.to_datetime(gdp.DATE).dt.quarter
gdp['year']=pd.to_datetime(gdp.DATE).dt.year
gdp.drop('DATE',axis=1,inplace=True)

yahoo_data=pd.merge(yahoo_data,dff,left_on=pd.to_datetime(yahoo_data.Date),right_on=pd.to_datetime(dff.DATE),how='inner')
yahoo_data=pd.merge(yahoo_data,unrate,on=['month','year'],how='inner')
#yahoo_data=pd.merge(yahoo_data,gdp,on=['quarter','year'],how='inner')
yahoo_data.drop(['key_0','DATE','month','quarter','year'],axis=1,inplace=True)
yahoo_data

,open,high,low,close,volume,Date,ticker,prec,temp,DFF,UNRATE
0,0.000000,62.580002,61.720001,62.320000,3700000,1962-12-11,^GSPC,2.26,31.5,2.88,5.5
1,0.000000,63.160000,62.130001,62.630001,3760000,1962-12-12,^GSPC,2.26,31.5,2.88,5.5
2,0.000000,63.070000,62.090000,62.419998,3380000,1962-12-13,^GSPC,2.26,31.5,2.88,5.5
3,0.000000,62.830002,61.959999,62.570000,3280000,1962-12-14,^GSPC,2.26,31.5,3.00,5.5
4,0.000000,62.950001,62.139999,62.369999,3590000,1962-12-17,^GSPC,2.26,31.5,3.00,5.5
...,...,...,...,...,...,...,...,...,...,...,...
14819,4553.689941,4572.620117,4537.359863,4566.479980,3250210000,2021-10-25,^GSPC,5.26,62,0.08,4.6
14820,4578.689941,4598.529785,4569.169922,4574.790039,2866500000,2021-10-26,^GSPC,5.26,62,0.08,4.6
14821,4580.220215,4584.569824,4551.660156,4551.680176,3259510000,2021-10-27,^GSPC,5.26,62,0.08,4.6
14822,4562.839844,4597.549805,4562.839844,4596.419922,3197560000,2021-10-28,^GSPC,5.26,62,0.08,4.6


In [22]:
# 1 year
days_out=252

# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html
# https://stackoverflow.com/questions/61319814/moving-average-in-pandas-issue-with-first-and-last-rows

yahoo_data['volume_moving'] = yahoo_data['volume'].rolling(days_out).mean().shift(periods=1)
yahoo_data['volume_moving_std']=yahoo_data['volume'].rolling(days_out).std().shift(periods=1)
yahoo_data['close_moving'] = yahoo_data['close'].rolling(days_out).mean().shift(periods=1)
yahoo_data['close_moving_std']=yahoo_data['close'].rolling(days_out).std().shift(periods=1)

# https://stackoverflow.com/questions/42138357/pandas-rolling-slope-calculation

def calc_slope(x):
    slope = np.polyfit(range(len(x)), x, 1)[0]
    return slope

yahoo_data['volume_slope'] = yahoo_data['volume'].rolling(days_out).apply(calc_slope).shift(periods=1)
yahoo_data['close_slope'] = yahoo_data['close'].rolling(days_out).apply(calc_slope).shift(periods=1)

yahoo_data['close_future'] = yahoo_data['close'].shift(periods=-days_out)

yahoo_data.drop(columns=['high','low','volume','ticker','Date'],inplace=True)
yahoo_data.dropna(inplace=True)
yahoo_data

,open,close,prec,temp,DFF,UNRATE,volume_moving,volume_moving_std,close_moving,close_moving_std,volume_slope,close_slope,close_future
252,0.000000,73.910004,2.31,31.2,3.50,5.5,4.519881e+06,9.292337e+05,69.227460,3.254833,4.207596e+03,0.041764,83.660004
253,0.000000,74.059998,2.31,31.2,3.50,5.5,4.521944e+06,9.279823e+05,69.273452,3.238684,4.101602e+03,0.041550,83.449997
254,0.000000,74.300003,2.31,31.2,3.50,5.5,4.524048e+06,9.268485e+05,69.318809,3.225283,4.007497e+03,0.041370,83.220001
255,0.000000,74.739998,2.31,31.2,3.50,5.5,4.527619e+06,9.241526e+05,69.365952,3.210835,3.876007e+03,0.041184,83.550003
256,0.000000,74.629997,2.31,31.2,3.50,5.5,4.535000e+06,9.215726e+05,69.414246,3.199715,3.815290e+03,0.041045,83.900002
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14567,3464.899902,3465.389893,5.05,57.9,0.09,6.9,4.712277e+09,1.420226e+09,3129.904879,258.544825,2.306975e+06,1.177150,4566.479980
14568,3441.419922,3400.969971,5.05,57.9,0.09,6.9,4.712094e+09,1.420360e+09,3131.710831,259.294343,2.109958e+06,1.197375,4574.790039
14569,3403.149902,3390.679932,5.05,57.9,0.09,6.9,4.714545e+09,1.418567e+09,3133.212497,259.755008,1.914548e+06,1.212360,4551.680176
14570,3342.479980,3271.030029,5.05,57.9,0.09,6.9,4.716235e+09,1.417393e+09,3134.606386,260.191789,1.729118e+06,1.227693,4596.419922


## -------------PRACTICE-------------
1.

In [23]:
yahoo_data.to_csv('yahoo_data.csv')

In [24]:
%ls

DFF.csv                    SP500_info.csv
Data_Cleaning-Copy1.ipynb  UNRATE.csv
Data_Cleaning.ipynb        constituents_csv.csv
GDPC1.csv                  nyc_precip.xlsx
Machine_Learning.ipynb     nyc_temp.xlsx
Regression.ipynb           yahoo_data.csv


<a id='55'></a>
# Weekly Readings/Videos

https://blog.trinket.io/why-python/
    
https://towardsdatascience.com/top-16-python-applications-in-real-world-a0404111ac23

<a id='56'></a>
# Extra Practice